# An intro to Matryoskha Embeddings

In [ ]:
%pip install ranx duckdb

In [1]:
import duckdb

In [2]:
con = duckdb.connect("olympics.duckdb")

In [3]:
con.sql("DESCRIBE olympics")


┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ index       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ text        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ url         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ title       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [4]:
con.sql("SELECT * FROM olympics LIMIT 10")


┌───────┬──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────┐
│ index │         text         │         url          │                             title                              │
│ int64 │       varchar        │       varchar        │                            varchar                             │
├───────┼──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────┤
│     0 │ The 2024 Olympics …  │ https://www.bbc.co…  │ Paris 2024 Olympic opening ceremony kicks off Games in uniqu…  │
│     1 │ Swapping a stadium…  │ https://www.bbc.co…  │ Paris 2024 Olympic opening ceremony kicks off Games in uniqu…  │
│     2 │ Blue, white and re…  │ https://www.bbc.co…  │ Paris 2024 Olympic opening ceremony kicks off Games in uniqu…  │
│     3 │ There were surpris…  │ https://www.bbc.co…  │ Paris 2024 Olympic opening ceremony kicks off Games in uniqu…  │
│     4 │ The day had starte…  

## Create Embeddings

In [5]:
con.sql("""
ALTER TABLE olympics 
ADD COLUMN embeddings_1024 FLOAT[1024];
""")

In [6]:
import llama_cpp

In [7]:
llm = llama_cpp.Llama(
  model_path="./models/mxbai-embed-large-v1-f16.gguf", 
  embedding=True, 
  verbose=False
)

In [8]:
rows = con.sql("SELECT index, text FROM olympics").fetchall()

In [9]:
embeddings = llm.create_embedding([text for index, text in rows])['data']

In [10]:
for (index, text), embedding in zip(rows, embeddings):
  con.execute(
    "UPDATE olympics SET embeddings_1024 = ? WHERE index = ?", 
    [embedding['embedding'], index]
  )

In [11]:
con.sql("SELECT * FROM olympics LIMIT 10""")


┌───────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────────────────────────────────────┐
│ index │         text         │         url          │        title         │             embeddings_1024             │
│ int64 │       varchar        │       varchar        │       varchar        │               float[1024]               │
├───────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────────────────────────┤
│     0 │ The 2024 Olympics …  │ https://www.bbc.co…  │ Paris 2024 Olympic…  │ [-0.65678865, 0.375833, -0.2038792, 0…  │
│     1 │ Swapping a stadium…  │ https://www.bbc.co…  │ Paris 2024 Olympic…  │ [-0.3371891, 0.25818387, 0.2695669, 0…  │
│     2 │ Blue, white and re…  │ https://www.bbc.co…  │ Paris 2024 Olympic…  │ [-0.83171785, 0.1003966, -0.060408145…  │
│     3 │ There were surpris…  │ https://www.bbc.co…  │ Paris 2024 Olympic…  │ [-0.5007831, -0.34818622, -0.252199, …  │
│     4 │ The day had starte…  

In [12]:
import numpy as np

## Create truncated embeddings

In [13]:
def normalize(vec: list[float]) -> list[float]:
    return (vec / np.linalg.norm(vec)).tolist()

In [14]:
con.create_function(name="normalize", function=normalize)

In [15]:
con.sql("""
SELECT index, text, 
       embeddings_1024, 
       normalize(embeddings_1024[:512])::float[512] AS embeddings_512
FROM olympics 
LIMIT 5
""")


┌───────┬──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────┐
│ index │         text         │   embeddings_1024    │                         embeddings_512                         │
│ int64 │       varchar        │     float[1024]      │                           float[512]                           │
├───────┼──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────────┤
│     0 │ The 2024 Olympics …  │ [-0.65678865, 0.37…  │ [-0.05296788, 0.030309716, -0.016442198, 0.010034871, -0.023…  │
│     1 │ Swapping a stadium…  │ [-0.3371891, 0.258…  │ [-0.030085044, 0.023035955, 0.024051582, 0.048912287, -0.044…  │
│     2 │ Blue, white and re…  │ [-0.83171785, 0.10…  │ [-0.07096644, 0.008566354, -0.0051543335, 0.056356296, -0.01…  │
│     3 │ There were surpris…  │ [-0.5007831, -0.34…  │ [-0.039365005, -0.02736984, -0.019824581, 0.03224219, -0.057…  │
│     4 │ The day had starte…  

In [16]:
dimensions = [16, 32, 64, 128, 256, 512]
for dimension in dimensions:
  con.sql(f"""
  ALTER TABLE olympics 
  ADD COLUMN embeddings_{dimension} FLOAT[{dimension}];
  """)

  con.sql(f"""
  UPDATE olympics 
  SET embeddings_{dimension} = normalize(embeddings_1024[:{dimension}])
  """)

In [17]:
def vector_search(query, dimension=1024):
  raw_embedding = llm.create_embedding([query])
  search_vector = raw_embedding['data'][0]['embedding']

  if dimension < len(search_vector):
    search_vector = normalize(search_vector[:dimension])

  return con.sql(f"""
  SELECT index, text,
          array_cosine_similarity(
            "embeddings_{dimension}", $searchVector::FLOAT[{dimension}]
          ) AS score
  FROM olympics
  ORDER BY score DESC
  LIMIT 3
  """, params={"searchVector": search_vector})

## Query embeddings

In [18]:
query = 'Where did the opening ceremony take place?'
vector_search(query)


┌───────┬──────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ index │                                               text                                               │   score   │
│ int64 │                                             varchar                                              │   float   │
├───────┼──────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┤
│     0 │ The 2024 Olympics opened in Paris in spectacular style with thousands of athletes sailing alon…  │ 0.6873059 │
│    29 │ The peace anthem, part of all Olympic opening ceremonies, is aligned with the message of unity…  │ 0.6783502 │
│    10 │ When organisers first revealed plans to hold the opening ceremony along the river in the heart…  │ 0.6705272 │
└───────┴──────────────────────────────────────────────────────────────────────────────────────────────────┴───────────┘

In [19]:
from rich.console import Console
c = Console()

In [20]:
with c.pager(styles=True):
  dimensions = [16, 32, 64, 128, 256, 512, 1024]
  for dimension in dimensions[::-1]:
    c.print(dimension)
    c.print(vector_search(query, dimension))

1024
┌───────┬──────────────────────────────────────────────────────────────────────────────────────────────────┬───────
────┐
│ index │                                               text                                               │   
score   │
│ int64 │                                             varchar                                              │   
float   │
├───────┼──────────────────────────────────────────────────────────────────────────────────────────────────┼───────
────┤
│     0 │ The 2024 Olympics opened in Paris in spectacular style with thousands of athletes sailing alon…  │ 
0.6873059 │
│    29 │ The peace anthem, part of all Olympic opening ceremonies, is aligned with the message of unity…  │ 
0.6783502 │
│    10 │ When organisers first revealed plans to hold the opening ceremony along the river in the heart…  │ 
0.6705272 │
└───────┴──────────────────────────────────────────────────────────────────────────────────────────────────┴───────
────┘

512
┌───────┬─────

In [24]:
from ranx import Qrels, Run, compare
from functools import partial

## Evalute embeddings

In [25]:
qrels = Qrels.from_file("data/questions.json")

In [26]:
functions = [
  (partial(vector_search, dimension=dimension), dimension)
  for dimension in dimensions
]

In [27]:
def create_run(qrels, retrieval_fn, name):
  run_dict = {
    question: {
      str(index): score
      for index, score in (retrieval_fn(question)
                            .select("index, score")
                            .fetchall()
                          )
    }
    for question in qrels.to_dict()
  }
  return Run(run_dict, name=name)

In [28]:
%%time
runs = [
   create_run(qrels, fn, name)
   for fn, name in functions
]

CPU times: user 2min 21s, sys: 3.56 s, total: 2min 25s
Wall time: 21.8 s


In [29]:
comparison = compare(
    qrels,
    runs=runs,
    metrics=["hit_rate"],
)

/Users/markhneedham/miniforge3/lib/python3.10/site-packages/ranx/metrics/hit_rate.py:38: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _hit_rate(qrels[i], run[i], k, rel_lvl)


In [30]:
comparison


#      Model  Hit Rate
---  -------  ----------
a         16  0.550
b         32  0.350
c         64  0.700ᵇ
d        128  0.850ᵇ
e        256  0.800ᵇ
f        512  0.950ᵃᵇ
g       1024  0.750ᵇ

In [31]:
with c.pager(styles=True):
  c.print(comparison.win_tie_loss)

{
    (16, 32): {'hit_rate': {'W': 5, 'T': 14, 'L': 1}},
    (16, 64): {'hit_rate': {'W': 3, 'T': 11, 'L': 6}},
    (16, 128): {'hit_rate': {'W': 2, 'T': 10, 'L': 8}},
    (16, 256): {'hit_rate': {'W': 3, 'T': 9, 'L': 8}},
    (16, 512): {'hit_rate': {'W': 1, 'T': 10, 'L': 9}},
    (16, 1024): {'hit_rate': {'W': 3, 'T': 10, 'L': 7}},
    (32, 16): {'hit_rate': {'W': 1, 'T': 14, 'L': 5}},
    (32, 64): {'hit_rate': {'W': 0, 'T': 13, 'L': 7}},
    (32, 128): {'hit_rate': {'W': 1, 'T': 8, 'L': 11}},
    (32, 256): {'hit_rate': {'W': 1, 'T': 9, 'L': 10}},
    (32, 512): {'hit_rate': {'W': 0, 'T': 8, 'L': 12}},
    (32, 1024): {'hit_rate': {'W': 1, 'T': 10, 'L': 9}},
    (64, 16): {'hit_rate': {'W': 6, 'T': 11, 'L': 3}},
    (64, 32): {'hit_rate': {'W': 7, 'T': 13, 'L': 0}},
    (64, 128): {'hit_rate': {'W': 1, 'T': 15, 'L': 4}},
    (64, 256): {'hit_rate': {'W': 1, 'T': 16, 'L': 3}},
    (64, 512): {'hit_rate': {'W': 0, 'T': 15, 'L': 5}},
    (64, 1024): {'hit_rate': {'W': 2, 'T': 15, 'L':